# unigram / bow / (yes/no filter)

In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from enum import Enum
from pprint import pprint
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import Phrases

import pyLDAvis
import pyLDAvis.gensim
from gensim.models.ldamodel import LdaModel

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.DEBUG)

2019-11-04 19:21:53,411 : DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## 自定義 data types and functions

In [2]:
class ContentType(Enum):
    TIT = 'title'
    ABS = 'abstract'
    AUT = 'author'
    SEC = 'section'
    
def get_contents(content_type):
    all_contents = []
    dataset_path = '../dataset'
    for file in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, file)
        if os.path.isfile(file_path):
            with open(file_path) as f:  
                line = f.readlines()
                if content_type == ContentType.AUT:
                    line = line[1]
                elif content_type == ContentType.SEC:
                    line = line[2]
                elif content_type == ContentType.ABS:
                    line = line[3]
                else:
                    line = line[0]
                line = line.strip()
                all_contents.append(line)
        else:
            print(file_path + ' does not exist.')
    return all_contents


def get_all_titles():
    return get_contents(ContentType.TIT)

def get_all_authors():        
    return get_contents(ContentType.AUT)

def get_all_sections():
    return get_contents(ContentType.SEC)

def get_all_abstracts():
    return get_contents(ContentType.ABS)

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            token = wordnet_lemmatizer.lemmatize(token, pos='v')
            token = wordnet_lemmatizer.lemmatize(token, pos='n')
            result.append(token)
    return result

def get_chart_data(num_topics,num_words,topics):
    buff = 300
    K = num_topics
    topicWordProbMat = topics
    
    #columns = ['1','2','3','4','5']
    columns = range(1,num_topics+1)

    df = pd.DataFrame(columns = columns)
    pd.set_option('display.width', 1000)

    # 40 will be resized later to match number of words in DC
    zz = np.zeros(shape=(buff,K))

    last_number = 0
    DC = {}

    for x in range (num_words): #取每個topic前10個字
        data= pd.DataFrame(columns=columns,index=[0])
        for i in range(num_topics):
            data[columns[i]] = ""
        df = df.append(data,ignore_index=True)  

    for line in topicWordProbMat:
        topic_id,words = line #一個line是一個topic
        probs = words.split("+")
        y = 0 #用來算第幾個word
        for pr in probs:    
            a = pr.split("*")
            df.iloc[y,topic_id] = a[1] #該word

            if a[1] in DC:
                zz[DC[a[1]]][topic_id] = a[0] #該word的機率
            else:
                zz[last_number][topic_id] = a[0]
                DC[a[1]] = last_number
                last_number = last_number+1
            y = y + 1

    return (df,DC,zz)

def show_words_table(df):  
    print(df)
    print('\n')
    
def show_dictionary(DC):
    print(DC)
    print('字典字數：',len(DC))
    print('\n')

def show_probs_table(zz):
    print(zz)
    print(zz.shape)
    
def show_heapmap(DC,zz):
    %matplotlib inline

    zz = np.resize(zz,(len(DC.keys()),zz.shape[1]))

    for val, key in enumerate(DC.keys()):
            plt.text(-3.5, val + 0.1, key,
                     horizontalalignment='right',
                     verticalalignment='center'
                     )

    #plt.figure(figsize=(10,50))
    plt.imshow(zz, cmap='hot', interpolation='nearest',aspect=0.5)#'auto'
    plt.show()

### 取出所有摘要

In [3]:
contents = get_all_abstracts()
print('共',len(contents),'篇論文\n')

documents = pd.DataFrame(data=contents,columns=['abstract'])
documents['index'] = documents.index
documents[:10]

共 1343 篇論文



,abstract,index
0,We consider the problem of actively eliciting ...,0
1,We investigate the task of distractor generati...,1
2,The most common representation formalisms for ...,2
3,Statistical relational learning models are pow...,3
4,Multimodal representation learning is gaining ...,4
5,Reinforcement learning (RL) has shown its adva...,5
6,Selecting appropriate tutoring help actions th...,6
7,Recognizing time expressions is a fundamental ...,7
8,"When facing large-scale image datasets, online...",8
9,Temporal modeling in videos is a fundamental y...,9


### 預處理的全部論文摘要

In [4]:
processed_docs = documents['abstract'].map(preprocess)
processed_docs[:10]

0    [consider, problem, actively, elicit, preferen...
1    [investigate, task, distractor, generation, mu...
2    [common, representation, formalism, plan, desc...
3    [statistical, relational, learn, model, powerf...
4    [multimodal, representation, learn, gain, deep...
5    [reinforcement, learn, show, advantage, image,...
6    [select, appropriate, tutor, help, action, acc...
7    [recognize, time, expression, fundamental, imp...
8    [face, large, scale, image, datasets, online, ...
9    [temporal, model, video, fundamental, challeng...
Name: abstract, dtype: object

## Dataset

### 產生字典

In [5]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('共',len(dictionary),'個字\n')

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
#dictionary.filter_extremes(no_below=10, no_above=0.2)
#print('Number of unique words after removing rare and common words:', len(dictionary))

2019-11-04 19:24:10,473 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-11-04 19:24:10,617 : INFO : built Dictionary(6927 unique tokens: ['active', 'actively', 'adaptive', 'aggregation', 'algorithm']...) from 1343 documents (total 121739 corpus positions)


共 6927 個字



### 產生 bag of words corpus

In [6]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('共',len(bow_corpus),'筆')

共 1343 筆


## Train LDA models

In [7]:
num_topics = 10
num_words = 10
passes = 30
iterations = 300

### Running LDA using BOW

In [8]:
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=num_topics, 
                                       id2word=dictionary, 
                                       passes=passes,
                                       iterations=iterations,
                                       eval_every=1)

2019-11-04 19:25:28,805 : INFO : using symmetric alpha at 0.1
2019-11-04 19:25:28,806 : INFO : using symmetric eta at 0.1
2019-11-04 19:25:28,811 : INFO : using serial LDA version on this node
2019-11-04 19:25:28,824 : INFO : running online LDA training, 10 topics, 30 passes over the supplied corpus of 1343 documents, updating every 6000 documents, evaluating every ~1343 documents, iterating 300x with a convergence threshold of 0.001000
2019-11-04 19:25:28,825 : INFO : training LDA model using 3 processes
2019-11-04 19:25:28,841 : DEBUG : worker process entering E-step loop
2019-11-04 19:25:28,852 : DEBUG : worker process entering E-step loop
2019-11-04 19:25:28,866 : DEBUG : getting a new job
2019-11-04 19:25:28,869 : DEBUG : getting a new job
2019-11-04 19:25:28,885 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-04 19:25:28,866 : DEBUG : worker process entering E-step loop
2019-11-04 19:25:28,884 : DEBUG : getting a new j

2019-11-04 19:25:38,961 : INFO : topic diff=0.125609, rho=0.462671
2019-11-04 19:25:38,980 : DEBUG : bound: at document #0
2019-11-04 19:25:40,113 : INFO : -7.450 per-word bound, 174.9 perplexity estimate based on a held-out corpus of 1343 documents with 121739 words
2019-11-04 19:25:40,114 : INFO : PROGRESS: pass 4, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-04 19:25:40,169 : DEBUG : processing chunk #0 of 1343 documents
2019-11-04 19:25:40,172 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-04 19:25:40,981 : DEBUG : 1338/1343 documents converged within 300 iterations
2019-11-04 19:25:40,986 : DEBUG : processed chunk, queuing the result
2019-11-04 19:25:40,991 : DEBUG : result put
2019-11-04 19:25:40,994 : DEBUG : updating topics
2019-11-04 19:25:40,995 : DEBUG : getting a new job
2019-11-04 19:25:41,004 : INFO : topic #9 (0.100): 0.021*"learn" + 0.015*"domain" + 0.014*"propose" + 0.013*"model" + 0.012*"method" + 0.012*"task"

2019-11-04 19:25:48,163 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-04 19:25:48,978 : DEBUG : 1340/1343 documents converged within 300 iterations
2019-11-04 19:25:48,983 : DEBUG : processed chunk, queuing the result
2019-11-04 19:25:48,987 : DEBUG : updating topics
2019-11-04 19:25:48,986 : DEBUG : result put
2019-11-04 19:25:48,988 : DEBUG : getting a new job
2019-11-04 19:25:48,994 : INFO : topic #6 (0.100): 0.012*"method" + 0.012*"learn" + 0.009*"image" + 0.007*"propose" + 0.007*"information" + 0.007*"approach" + 0.007*"task" + 0.007*"result" + 0.007*"model" + 0.006*"base"
2019-11-04 19:25:48,995 : INFO : topic #5 (0.100): 0.019*"network" + 0.016*"feature" + 0.014*"propose" + 0.012*"view" + 0.012*"shape" + 0.010*"model" + 0.009*"object" + 0.009*"method" + 0.007*"neural" + 0.007*"learn"
2019-11-04 19:25:48,995 : INFO : topic #8 (0.100): 0.020*"model" + 0.013*"propose" + 0.010*"learn" + 0.008*"approach" + 0.007*"event" + 0.007*"result" + 0.007*"task" + 0.006*"a

2019-11-04 19:25:58,643 : INFO : topic #5 (0.100): 0.020*"network" + 0.016*"feature" + 0.014*"propose" + 0.012*"shape" + 0.012*"view" + 0.012*"object" + 0.009*"method" + 0.009*"model" + 0.008*"image" + 0.007*"neural"
2019-11-04 19:25:58,643 : INFO : topic #1 (0.100): 0.011*"problem" + 0.010*"propose" + 0.008*"learn" + 0.008*"decision" + 0.008*"approach" + 0.008*"rule" + 0.007*"network" + 0.007*"base" + 0.006*"method" + 0.005*"model"
2019-11-04 19:25:58,645 : INFO : topic #6 (0.100): 0.012*"method" + 0.011*"learn" + 0.010*"image" + 0.007*"information" + 0.007*"approach" + 0.007*"result" + 0.007*"propose" + 0.007*"task" + 0.006*"base" + 0.006*"model"
2019-11-04 19:25:58,646 : INFO : topic #7 (0.100): 0.020*"learn" + 0.017*"propose" + 0.016*"method" + 0.015*"feature" + 0.010*"network" + 0.009*"model" + 0.008*"cluster" + 0.008*"video" + 0.008*"sample" + 0.008*"data"
2019-11-04 19:25:58,648 : INFO : topic diff=0.041467, rho=0.270453
2019-11-04 19:25:58,658 : DEBUG : bound: at document #0
20

2019-11-04 19:26:06,883 : INFO : topic #1 (0.100): 0.010*"propose" + 0.010*"problem" + 0.008*"decision" + 0.008*"rule" + 0.008*"approach" + 0.008*"learn" + 0.007*"network" + 0.007*"base" + 0.006*"method" + 0.005*"test"
2019-11-04 19:26:06,885 : INFO : topic diff=0.031655, rho=0.237883
2019-11-04 19:26:06,894 : DEBUG : bound: at document #0
2019-11-04 19:26:07,952 : INFO : -7.379 per-word bound, 166.5 perplexity estimate based on a held-out corpus of 1343 documents with 121739 words
2019-11-04 19:26:07,953 : INFO : PROGRESS: pass 17, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-04 19:26:08,200 : DEBUG : processing chunk #0 of 1343 documents
2019-11-04 19:26:08,202 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-04 19:26:08,902 : DEBUG : 1341/1343 documents converged within 300 iterations
2019-11-04 19:26:08,918 : DEBUG : processed chunk, queuing the result
2019-11-04 19:26:08,920 : DEBUG : result put
2019-11-04 19:26:08,923 : DEB

2019-11-04 19:26:15,592 : INFO : PROGRESS: pass 21, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-04 19:26:15,627 : DEBUG : processing chunk #0 of 1343 documents
2019-11-04 19:26:15,629 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-04 19:26:16,317 : DEBUG : 1343/1343 documents converged within 300 iterations
2019-11-04 19:26:16,323 : DEBUG : processed chunk, queuing the result
2019-11-04 19:26:16,327 : DEBUG : updating topics
2019-11-04 19:26:16,325 : DEBUG : result put
2019-11-04 19:26:16,328 : DEBUG : getting a new job
2019-11-04 19:26:16,334 : INFO : topic #5 (0.100): 0.021*"network" + 0.017*"object" + 0.016*"feature" + 0.014*"propose" + 0.013*"shape" + 0.012*"view" + 0.011*"image" + 0.009*"method" + 0.008*"point" + 0.007*"model"
2019-11-04 19:26:16,335 : INFO : topic #8 (0.100): 0.020*"model" + 0.012*"propose" + 0.009*"event" + 0.008*"learn" + 0.007*"approach" + 0.007*"user" + 0.007*"result" + 0.006*"work" + 0.006*"base" + 

2019-11-04 19:26:23,294 : DEBUG : updating topics
2019-11-04 19:26:23,294 : DEBUG : getting a new job
2019-11-04 19:26:23,301 : INFO : topic #8 (0.100): 0.021*"model" + 0.012*"propose" + 0.010*"event" + 0.008*"learn" + 0.007*"user" + 0.007*"approach" + 0.006*"work" + 0.006*"result" + 0.006*"base" + 0.006*"task"
2019-11-04 19:26:23,302 : INFO : topic #7 (0.100): 0.020*"learn" + 0.019*"propose" + 0.018*"method" + 0.016*"feature" + 0.011*"network" + 0.009*"model" + 0.009*"data" + 0.008*"sample" + 0.008*"cluster" + 0.008*"video"
2019-11-04 19:26:23,304 : INFO : topic #3 (0.100): 0.020*"model" + 0.019*"text" + 0.014*"sentence" + 0.011*"propose" + 0.011*"learn" + 0.011*"word" + 0.010*"generate" + 0.010*"language" + 0.010*"train" + 0.009*"task"
2019-11-04 19:26:23,305 : INFO : topic #0 (0.100): 0.031*"model" + 0.017*"network" + 0.014*"propose" + 0.011*"learn" + 0.011*"method" + 0.010*"neural" + 0.009*"train" + 0.008*"base" + 0.007*"task" + 0.007*"state"
2019-11-04 19:26:23,307 : INFO : topic 

2019-11-04 19:26:29,937 : INFO : topic #4 (0.100): 0.018*"algorithm" + 0.018*"agent" + 0.017*"problem" + 0.011*"learn" + 0.010*"approach" + 0.010*"model" + 0.008*"base" + 0.008*"time" + 0.007*"game" + 0.007*"plan"
2019-11-04 19:26:29,939 : INFO : topic #0 (0.100): 0.031*"model" + 0.017*"network" + 0.014*"propose" + 0.011*"learn" + 0.011*"method" + 0.011*"neural" + 0.009*"train" + 0.008*"base" + 0.007*"task" + 0.007*"state"
2019-11-04 19:26:29,940 : INFO : topic #7 (0.100): 0.020*"learn" + 0.019*"propose" + 0.018*"method" + 0.016*"feature" + 0.010*"network" + 0.009*"model" + 0.009*"data" + 0.009*"sample" + 0.008*"cluster" + 0.008*"image"
2019-11-04 19:26:29,941 : INFO : topic diff=0.016271, rho=0.180565
2019-11-04 19:26:29,950 : DEBUG : bound: at document #0
2019-11-04 19:26:30,937 : INFO : -7.357 per-word bound, 164.0 perplexity estimate based on a held-out corpus of 1343 documents with 121739 words


In [9]:
vis_data = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
# 在 notebook 中显示可视化结果，需要调用 display 方法，或者执行 “pyLDAvis.enable_notebook()” ，即可在 notebook 中自动展示可视化结果，无需再调用 display
pyLDAvis.display(vis_data)

2019-11-04 17:02:11,455 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-04 17:02:12,052 : DEBUG : 1343/1343 documents converged within 300 iterations
/Users/alexis/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [10]:
file_name = '../models/lda_unigram_bow_topic_'+ str(num_topics) +'.model'
print(file_name)
lda_model.save(file_name)

2019-11-04 17:02:49,976 : INFO : saving LdaState object under ../models/lda_unigram_bow_topic_10.model.state, separately None
2019-11-04 17:02:49,978 : WARNING : this function is deprecated, use smart_open.open instead
/Users/alexis/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
2019-11-04 17:02:49,982 : DEBUG : {'uri': '../models/lda_unigram_bow_topic_10.model.state', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': {}}
2019-11-04 17:02:49,991 : INFO : saved ../models/lda_unigram_bow_topic_10.model.state
2019-11-04 17:02:49,997 : WARNING : this function is deprecated, use smart_open.open instead
/Users/alexis/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: Dep

../models/lda_unigram_bow_topic_10.model
